# Multiclass SVM Development Code

This notebook will contain the development portion for the traditional model being used, which is the Support Vector Machine or SVM.

## Methodology

To choose the final model, the methodology is as follows:

### 1. Load and preprocess the images

There are three main methods to preprocess the images: the traditional, the CNN, and the transformer method. The **traditional method** uses a mix of different preprocessing techniques like histogram equilization (CLAHE), noise reduction (Gaussian Blur), resizing, normalization, grayscaling, and others before being passed to texture analysis algorithms. The **CNN** method will use the appropriate preprocessing function for that network, which will then be passed onto a pre-trained CNN for feature extraction. Similar to the CNN method, the **transformer** method will use the proper preprocessing function and then will pass the image to the pre-trained vision transformer for feature extraction.

For the traditional method, Local Binary Pattern will be used as the texture analysis algorithm. For the CNN method, VGG-16 will be used as the pre-trained neural network. DINOv2 will be used for the transformer method. All methods will be used and passed to the SVM model for comparison.

### 2. Train and Evaluate the Model

From the two different preprocessing techniques, both will be passed into an SVM for comparison. The model will be trained using K-fold cross validation and will be evaluated using accuracy and macro f1 score. K-fold CV is used to ensure that the model is reliable and is generalized across different data splits. Accuracy is used since we want to identify the proportion of correctly predicted images. That is, we want to accurately classify the images, which makes accuracy a good metric. Macro f1 score is also used as it evaluates the f1 score for each class. This makes it a good metric to evaluate the model's precision and recall even if there is an imbalance in the frequency of each class.

Note that a random forest model was also trained to compare the results between RF and SVM using the CNN and transformer feature extraction.

### 3. Hypertune

Based on the performance of each data preprocessing technique, the model that produces the better performance will be used for hypertuning. Grid Search CV will be used to hypertune the model to help find the best hyperparameters without overfitting the model. After which, the model will be evaluated again to check whether the performance of the model is better or worse after hypertuning.

**Note:** This notebook was ran using Google Colab. If the file was ran locally, change the directories to the proper ones.

## Miscellaneous

In [1]:
# %pip install -r requirements.txt -q

# To hide warnings produced by different packages
import warnings
warnings.filterwarnings('ignore')

## Imports

In [2]:
import cv2
import cv2.typing as cv_typing
from pathlib import Path
import os
import numpy as np
import pandas as pd
import typing
from PIL import Image
import requests
import tensorflow as tf

# Traditional image processing
from skimage.feature import local_binary_pattern

# Feature extraction
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from transformers import AutoImageProcessor, Dinov2Model
import torch

# Model training and testing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import classification_report, accuracy_score, f1_score, make_scorer

# Exporting the model
import pickle

# Time
import time
from tqdm import tqdm

# Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Checking existence of GPU

In [3]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Constants

In [38]:
classes = ["Bacteria", "Fungi", "Healthy", "Pest", "Phytopthora", "Virus"]

# Path of where the train images are located
img_dir = "/content/drive/MyDrive/BSCS/CS180/Project/potato_train"
# img_dir = "./data/potato_train" # if local

# Path for final model
model_dir = "/content/drive/MyDrive/BSCS/CS180/Project/models"
# model_dir = "./models" # if local

# Other constants
ORIG_IMG_SIZE = (1500,1500)
BATCH_SIZE = 8

In [6]:
# Set seed
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)

In [7]:
# VGG16 model for feature extraction
vgg_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

# CLAHE
clahe = cv2.createCLAHE(clipLimit=5.0)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [8]:
# Dinov2 processor and model
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-large", use_fast=True)
dino_model = Dinov2Model.from_pretrained("facebook/dinov2-large").to(device)
dino_model.eval()

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

`use_fast` is set to `True` but the image processor class does not have a fast version.  Falling back to the slow version.


config.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Dinov2Model(
  (embeddings): Dinov2Embeddings(
    (patch_embeddings): Dinov2PatchEmbeddings(
      (projection): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Dinov2Encoder(
    (layer): ModuleList(
      (0-23): 24 x Dinov2Layer(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attention): Dinov2Attention(
          (attention): Dinov2SelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (output): Dinov2SelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (layer_scale1): Dinov2LayerScale()
        (drop_path): Identity()
        (norm2): LayerNorm((1024,),

## Data (Image) Preprocessing

This section covers the first step of the methodology. This includes loading all the images from the directory, passing them through a series of image preprocessing techniques, and turning it into a final numpy array for the model to use as input.

### Loading Data

In [9]:
def get_all_images(
    file_path: str = "./potato_train",
    classes: list[str] = ["Bacteria", "Fungi", "Healthy", "Pest", "Phytopthora", "Virus"],
    resize_dim: tuple[int, int] = (518, 518),
    color: int = cv2.IMREAD_COLOR_RGB,
    trad: bool = True
) -> tuple[list[cv_typing.MatLike], np.ndarray[typing.Any, typing.Any]]:
    """
    Function to load all images

    Returns a tuple of images, y (labels/classes) of each image, and the label encoder
    """
    # Get folder
    dir = Path(file_path)

    # Check if directory
    if not dir.is_dir():
        raise Exception("Please enter a valid directory")

    # Get all subdirectories in the dir
    walk_dir = list(os.walk(dir))[1:]

    # List to hold the labels
    class_labels: list[int] = []

    # Variable for final array
    final_imgs: list[cv_typing.MatLike] = []

    # Go through each sub directory
    for sub_dir in walk_dir:
        try:
            # Get the name of the class of the image
            class_name = os.path.basename(sub_dir[0])

            # Get the appropriate numerical encoding for the class
            class_label = classes.index(class_name)

            # Get all images in that class
            imgs = [f"{sub_dir[0]}/{img_name}" for img_name in sub_dir[-1]]

            for img_path in imgs:
                if trad:
                    img_read: cv2.typing.MatLike = cv2.imread(img_path, color)
                    img_read = cv2.resize(img_read, resize_dim)
                    final_imgs.append(img_read)
                else:
                    img_loaded: Image.Image = image.load_img(img_path, target_size=ORIG_IMG_SIZE)
                    img_array: np.ndarray[typing.Any, typing.Any] = image.img_to_array(img_loaded)
                    img_array = cv2.resize(img_array, resize_dim)
                    final_imgs.append(img_array)

                class_labels.append(class_label)

            print(f"DONE: Folder {sub_dir[0]} with name {class_name} and {len(imgs)/len(sub_dir[-1]) * 100}% images loaded")
        except Exception as e:
            print(f"Failed on directory {sub_dir[0]}: {e}")

    return (final_imgs, np.array(class_labels))

### Traditional Method

To apply SVM, the image must first be preprocessed. The `preprocess_img_tm` function encompasses the different combinations of preprocessing techniques. Namely:
- resizing
- CLAHE (histogram equalization)
- blurring (gaussian blurring)
- local binary pattern (for texture analysis)

Note that the expected output should be a 1D array since an SVM can only accept 1D feature arrays

In [10]:
def preprocess_img_tm(
    img: cv_typing.MatLike,
    use_grayscale: bool = True,
    use_resize: bool = True,
    resize_dim: tuple[int, int] = (224, 224),
    use_clahe: bool = False,
    use_blurring: bool = False,
    blur_k_size: tuple[int, int] = (5,5),
    blur_sigma_x: float = 0.0,
    use_lbp: bool = False,
    lbp_radius: int = 1,
    lbp_points: int = 8,
    lbp_method: str = "uniform",
) -> cv_typing.MatLike:
    """Function that processes the input image"""

    # Grayscale image
    if use_grayscale:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Resize image
    if use_resize and resize_dim != (224,224):
        img = cv2.resize(img, resize_dim)

    # Apply CLAHE
    if use_clahe:
        img = clahe.apply(img)

    # Apply Gaussian Blur
    if use_blurring:
        img = cv2.GaussianBlur(img, blur_k_size, blur_sigma_x)

    # Apply LBP and histogram encoding
    if use_lbp:
        if not use_grayscale:
            # Grayscale the image if it hasn't been already
            # (pre-requisite of LBP)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        lbp_output = local_binary_pattern(img, lbp_points, lbp_radius, method=lbp_method)
        hist, _ = np.histogram(
            lbp_output.ravel(),
            bins=np.arange(0, lbp_points + 3),
            range=(0, lbp_points + 2)
        )
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        return hist  # 1D histogram

    # Return flattened image if no LBP
    return img.flatten()

### CNN and Transformer Method

The `preprocess_img_other` is a function that will preprocess the images using VGG16 or DINOv2. Generally, it will preprocess the input based on the needs of the model and then use the model to extract the features of the image.

In [11]:
def preprocess_img_other(
    img_array: np.ndarray[typing.Any, typing.Any] = np.empty((1,1)),
    imgs: list[np.ndarray[typing.Any, typing.Any]] = [],
    resize: bool = True,
    resize_dim: tuple[int, int] = (256,256),
    use_vgg: bool = True,
    use_dino: bool = False,
) -> np.ndarray[typing.Any, typing.Any]:
    feature: np.ndarray[typing.Any, typing.Any]

    try:
        if use_vgg:
            with tf.device('/device:GPU:0'):
                if resize:
                    # Resize the image
                    img_array = cv2.resize(img_array, resize_dim)

                    # Normalize pixel values
                    img_array = img_array / 255.0

                # Expand the dimensions of the array
                img_array = np.expand_dims(img_array, axis=0)
                print("\tExpanded the array")

                # Use the built-in pre-processing method for VGG16 (made by keras)
                img_preprocessed = preprocess_input(img_array)
                print("\tFinished pre-processing the input")

                # Use the VGG16 model to extract the features
                feature = vgg_model.predict(img_preprocessed, batch_size=BATCH_SIZE)[0]
                print("\tFeatures taken using VGG16\n")
        elif use_dino:
            all_features = []

            # Split into batches
            for i in tqdm(range(0, len(imgs), BATCH_SIZE)):
                batch = imgs[i:i+BATCH_SIZE]

                # Limit pixel values and convert each image to PIL
                batch = [Image.fromarray(np.clip(img, 0, 255).astype(np.uint8)) for img in batch]

                # Preprocess and move to GPU
                print(f"\tProcesing batch {i}")
                inputs = processor(images=batch, return_tensors="pt").to(device)
                print("\tFinished processing the batch")

                # Forward pass
                with torch.no_grad():
                    outputs = dino_model(**inputs)
                    features = outputs.pooler_output
                    print("\tFeatures taken using  DINOv2")

                all_features.append(features.cpu().numpy())
                print("\tCompleted all batches for  DINOv2\n")

            # Combine all batches into one
            return np.vstack(all_features)
    except RuntimeError as e:
        print(e)

    return feature

## Models Functionality

To make it easier to test SVM and RF on different kinds of inputs, a function will be made. This function simply takes in the X features and y labels and outputs the metrics when using that set of preprocessing techniques.

In [12]:
def run_train_cross_val(
    X: list[cv_typing.MatLike],
    y: np.ndarray[typing.Any, typing.Any],
    k_folds: int = 5,
    random_state: int = 42,
    method: str = "trad",
    model = SVC(kernel="rbf", C=10, gamma='scale'),
    **preproc_args
) -> dict:
    """
    Run a classifier with cross-validation using a preprocessing pipeline.
    Returns a dictionary with train and validation metrics.
    """

    # Preprocess images
    if method == "trad":
        print("Preprocessing image using Traditional Methods")
        X_preprocessed = np.array([preprocess_img_tm(img, **preproc_args) for img in X])
    elif method == "dino":
        print("Preprocessing image using DINOv2")
        X_preprocessed = preprocess_img_other(imgs=X, use_vgg=False, use_dino=True)
    elif method == "vgg":
        print("Preprocessing image using VGG16")
        X_preprocessed = np.array([preprocess_img_other(img_array=img) for img in tqdm(X)])
    else:
        return {"result": f"{method} is not a supported preprocessing method"}

    print("Done preprocessing images")

    # Define pipeline: scaler + SVM
    pipeline = make_pipeline(
        StandardScaler(),
        model
    )

    # Define stratified k-fold
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_state)

    # Define custom scorers
    scorers = {
        'accuracy': make_scorer(accuracy_score),
        'f1_macro': make_scorer(f1_score, average='macro')
    }

    print(f"Running {k_folds}-Fold Cross-Validation\n")
    start_time = time.time()

    # Run cross-validation
    results = cross_validate(
        estimator=pipeline,
        X=X_preprocessed,
        y=y,
        scoring=scorers,
        cv=skf,
        return_train_score=True
    )

    total_time = time.time() - start_time

    # Print summary
    print(f"\tAvg Train Acc:\t{np.mean(results['train_accuracy']):.4f}")
    print(f"\tAvg Val Acc:\t{np.mean(results['test_accuracy']):.4f}")
    print(f"\tAvg Train F1:\t{np.mean(results['train_f1_macro']):.4f}")
    print(f"\tAvg Val F1:\t{np.mean(results['test_f1_macro']):.4f}")
    print(f"\tTotal Time:\t{int(total_time//60)}m {total_time % 60:.2f}s")

    return {
        "train_accuracy": results['train_accuracy'],
        "val_accuracy": results['test_accuracy'],
        "train_f1": results['train_f1_macro'],
        "val_f1": results['test_f1_macro'],
        "time": total_time
    }

## Testing with Different Combinations

Before testing different image preprocessing combinations (either traditional or CNN), it is important to split the dataset into the train, test, and validation sets wherein the train will be used to fit the model (normal and hypertuned) and do a preliminary analysis of the final image preprocessing steps, validation will be used to evaluate the training and hypertuned model, and test will be used as a last step to evaluate the hypertuned model.

As such, only the train and validation sets are repeatedly used with different sets of image preprocessing techniques. The test set is only preprocessed once the hypertuned model is being evaluated.

For traditional

In [34]:
X_trad, y_trad = get_all_images(classes=classes, file_path=img_dir, resize_dim=(224,224), trad=True)

# Split into train_val and test
X_train_val_trad, X_test_trad, y_train_val_trad, y_test_trad = train_test_split(
    X_trad, y_trad, test_size=0.2, stratify=y_trad, random_state=42
)

print()
print(f"Total size of X (Trad) = {len(X_trad)} | Len of Train Val = {len(X_train_val_trad)} | Len of Test = {len(X_test_trad)} | Total = {len(X_train_val_trad) + len(X_test_trad)}")
print(f"Total size of y (Trad) = {len(y_trad)} | Len of Train Val = {len(y_train_val_trad)} | Len of Test = {len(y_test_trad)} | Total = {len(y_train_val_trad) + len(y_test_trad)}")

DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Pest with name Pest and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Healthy with name Healthy and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Fungi with name Fungi and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Virus with name Virus and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Bacteria with name Bacteria and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Phytopthora with name Phytopthora and 100.0% images loaded

Total size of X (Trad) = 2707 | Len of Train Val = 2165 | Len of Test = 542 | Total = 2707
Total size of y (Trad) = 2707 | Len of Train Val = 2165 | Len of Test = 542 | Total = 2707


For feature extraction using CNN/Vision Transformer

In [13]:
# Get the images and their labels
X_other, y_other = get_all_images(classes=classes, file_path=img_dir, trad=False)

X_train_val_other, X_test_other, y_train_val_other, y_test_other = train_test_split(
    X_other, y_other, test_size=0.2, stratify=y_other, random_state=42
)

print()
print(f"Total size of X (Others) = {len(X_other)} | Len of Train Val = {len(X_train_val_other)} | Len of Test = {len(X_test_other)} | Total = {len(X_train_val_other) + len(X_test_other)}")
print(f"Total size of y (Others) = {len(y_other)} | Len of Train Val = {len(y_train_val_other)} | Len of Test = {len(y_test_other)} | Total = {len(y_train_val_other) + len(y_test_other)}")

DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Pest with name Pest and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Healthy with name Healthy and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Fungi with name Fungi and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Virus with name Virus and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Bacteria with name Bacteria and 100.0% images loaded
DONE: Folder /content/drive/MyDrive/BSCS/CS180/Project/potato_train/Phytopthora with name Phytopthora and 100.0% images loaded

Total size of X (Others) = 2707 | Len of Train Val = 2165 | Len of Test = 542 | Total = 2707
Total size of y (Others) = 2707 | Len of Train Val = 2165 | Len of Test = 542 | Total = 2707


### Traditional Image Preprocessing

In [36]:
# Create a list of image preprocessing configurations (for the traditional method)
trad_configs = [
    {'use_resize': True, 'use_clahe': True},
    {'use_resize': True, 'resize_dim': (256,256), 'use_blurring': True},
    {'use_resize': True, 'resize_dim': (256,256), 'use_lbp': True},
    {'use_resize': True, 'use_clahe': True, 'use_blurring': True},
    {'use_resize': True, 'use_clahe': True, 'use_blurring': False, 'use_lbp': True},
    {'use_resize': True, 'use_clahe': False, 'use_blurring': True, 'use_lbp': True},
    {'use_resize': True, 'use_clahe': True, 'use_blurring': True, 'use_lbp': True},
    {'use_resize': True, 'resize_dim': (256,256), 'use_clahe': True, 'use_blurring': True, 'use_lbp': True},
]

# Use SVM and evaluate the model
results_per_config = []

for i, config in enumerate(trad_configs):
    print(f"===== RUNNING CONFIG {i+1}/{len(trad_configs)} =====")
    print(f"Image preprocessing arguments: {config}")

    result_trad = run_train_cross_val(
        X=X_train_val_trad,
        y=y_train_val_trad,
        method="trad",
        **config
    )

    # Create a flat result dictionary for DataFrame
    result_record = {
        "config_id": i,
        "config": config,
        "train_acc_mean": np.mean(result_trad["train_accuracy"]),
        "val_acc_mean": np.mean(result_trad["val_accuracy"]),
        "train_f1_mean": np.mean(result_trad["train_f1"]),
        "val_f1_mean": np.mean(result_trad["val_f1"]),
        "total_time": result_trad["time"]
    }

    results_per_config.append(result_record)

===== RUNNING CONFIG 1/8 =====
Image preprocessing arguments: {'use_resize': True, 'use_clahe': True}
Preprocessing image using Traditional Methods
Done preprocessing images
Running 5-Fold Cross-Validation

	Avg Train Acc:	0.9987
	Avg Val Acc:	0.3312
	Avg Train F1:	0.9990
	Avg Val F1:	0.2791
	Total Time:	39m 5.13s
===== RUNNING CONFIG 2/8 =====
Image preprocessing arguments: {'use_resize': True, 'resize_dim': (256, 256), 'use_blurring': True}
Preprocessing image using Traditional Methods
Done preprocessing images
Running 5-Fold Cross-Validation

	Avg Train Acc:	0.9987
	Avg Val Acc:	0.3857
	Avg Train F1:	0.9990
	Avg Val F1:	0.3466
	Total Time:	50m 20.82s
===== RUNNING CONFIG 3/8 =====
Image preprocessing arguments: {'use_resize': True, 'resize_dim': (256, 256), 'use_lbp': True}
Preprocessing image using Traditional Methods
Done preprocessing images
Running 5-Fold Cross-Validation

	Avg Train Acc:	0.5390
	Avg Val Acc:	0.4333
	Avg Train F1:	0.4932
	Avg Val F1:	0.3794
	Total Time:	0m 1.71s

In [37]:
# Create the DataFrame and show the results
df_results = pd.DataFrame(results_per_config)
display(df_results)

,config_id,config,train_acc_mean,val_acc_mean,train_f1_mean,val_f1_mean,total_time
0,0,"{'use_resize': True, 'use_clahe': True}",0.998730,0.331178,0.999005,0.279076,2345.133285
1,1,"{'use_resize': True, 'resize_dim': (256, 256),...",0.998730,0.385681,0.999006,0.346597,3020.822070
2,2,"{'use_resize': True, 'resize_dim': (256, 256),...",0.539030,0.433256,0.493199,0.379396,1.714239
3,3,"{'use_resize': True, 'use_clahe': True, 'use_b...",0.998730,0.370901,0.999006,0.319995,2319.525325
4,4,"{'use_resize': True, 'use_clahe': True, 'use_b...",0.503580,0.412471,0.443155,0.341694,1.739115
5,5,"{'use_resize': True, 'use_clahe': False, 'use_...",0.572517,0.439723,0.557994,0.417638,1.683848
6,6,"{'use_resize': True, 'use_clahe': True, 'use_b...",0.547229,0.434180,0.517457,0.383889,1.724345
7,7,"{'use_resize': True, 'resize_dim': (256, 256),...",0.544573,0.440647,0.510182,0.393085,1.700990


### Feature Extraction Preprocessing

**Test 1: SVM with DINOv2 Model**

In [14]:
result_dino_svm = run_train_cross_val(
    X=X_train_val_other,
    y=y_train_val_other,
    method="dino"
)

Preprocessing image using DINOv2


  0%|          | 0/271 [00:00<?, ?it/s]

	Procesing batch 0
	Finished processing the batch


  0%|          | 1/271 [00:01<05:44,  1.28s/it]

	Features taken using  DINOv2
	Completed all batches for  DINOv2

	Procesing batch 8


...

	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

	Procesing batch 2152


100%|█████████▉| 270/271 [01:06<00:00,  4.16it/s]

	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

	Procesing batch 2160


100%|██████████| 271/271 [01:06<00:00,  4.07it/s]


	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

Done preprocessing images
Running 5-Fold Cross-Validation

	Avg Train Acc:	0.9987
	Avg Val Acc:	0.8471
	Avg Train F1:	0.9990
	Avg Val F1:	0.8444
	Total Time:	0m 15.99s


**Test 2: Random Forest with DINOv2 Model**

In [15]:
result_dino_rf = run_train_cross_val(
    X=X_train_val_other,
    y=y_train_val_other,
    method="dino",
    model=RandomForestClassifier(random_state=42)
)

Preprocessing image using DINOv2


  0%|          | 0/271 [00:00<?, ?it/s]

	Procesing batch 0
	Finished processing the batch
	Features taken using  DINOv2


  0%|          | 1/271 [00:00<01:03,  4.22it/s]

	Completed all batches for  DINOv2

	Procesing batch 8
	Finished processing the batch
	Features taken using  DINOv2


...

 99%|█████████▉| 269/271 [01:04<00:00,  4.16it/s]

	Completed all batches for  DINOv2

	Procesing batch 2152
	Finished processing the batch
	Features taken using  DINOv2


100%|██████████| 271/271 [01:05<00:00,  4.15it/s]

	Completed all batches for  DINOv2

	Procesing batch 2160
	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

Done preprocessing images
Running 5-Fold Cross-Validation



	Avg Train Acc:	0.9987
	Avg Val Acc:	0.7658
	Avg Train F1:	0.9990
	Avg Val F1:	0.7543
	Total Time:	0m 24.41s


**Test 3: SVM with VGG16**

In [16]:
result_vgg_svm = run_train_cross_val(
    X=X_train_val_other,
    y=y_train_val_other,
    method="vgg"
)

Preprocessing image using VGG16


  0%|          | 0/2165 [00:00<?, ?it/s]

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


  0%|          | 1/2165 [00:03<2:01:21,  3.36s/it]

	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


...

100%|█████████▉| 2163/2165 [02:53<00:00, 12.39it/s]

	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


100%|██████████| 2165/2165 [02:53<00:00, 12.46it/s]

	Features taken using VGG16

Done preprocessing images
Running 5-Fold Cross-Validation



	Avg Train Acc:	0.8857
	Avg Val Acc:	0.4129
	Avg Train F1:	0.8743
	Avg Val F1:	0.3660
	Total Time:	0m 12.27s


**Test 4: Random Forest with VGG16**

In [17]:
result_vgg_rf = run_train_cross_val(
    X=X_train_val_other,
    y=y_train_val_other,
    method="vgg",
    model=RandomForestClassifier(random_state=42)
)

Preprocessing image using VGG16


  0%|          | 0/2165 [00:00<?, ?it/s]

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


  0%|          | 2/2165 [00:00<02:54, 12.39it/s]

	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


...

100%|█████████▉| 2164/2165 [02:53<00:00, 12.24it/s]

	Features taken using VGG16

	Expanded the array
	Finished pre-processing the input
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


100%|██████████| 2165/2165 [02:54<00:00, 12.44it/s]

	Features taken using VGG16

Done preprocessing images
Running 5-Fold Cross-Validation



	Avg Train Acc:	0.9987
	Avg Val Acc:	0.4166
	Avg Train F1:	0.9990
	Avg Val F1:	0.3522
	Total Time:	0m 3.81s


Compile results of feature extraction into one

In [33]:
result_per_test = []
other_tests = [result_dino_svm, result_dino_rf, result_vgg_svm, result_vgg_rf]

for other_test in other_tests:
    temp = {
        "train_acc_mean": np.round(np.mean(other_test['train_accuracy']), 4),
        "val_acc_mean": np.round(np.mean(other_test['val_accuracy']), 4),
        "train_f1_mean": np.round(np.mean(other_test['train_f1']), 4),
        "val_f1_mean": np.round(np.mean(other_test['val_f1']), 4),
        "total_time": f"{int(other_test['time']//60)}m {other_test['time'] % 60:.2f}s"
    }

    result_per_test.append(temp)

df_other_results = pd.DataFrame(result_per_test)
display(df_other_results)

,train_acc_mean,val_acc_mean,train_f1_mean,val_f1_mean,total_time
0,0.9987,0.8471,0.9990,0.8444,0m 15.99s
1,0.9987,0.7658,0.9990,0.7543,0m 24.41s
2,0.8857,0.4129,0.8743,0.3660,0m 12.27s
3,0.9987,0.4166,0.9990,0.3522,0m 3.81s


### Comparison of Results

**Summary of Results**

| Type of Config | Config + Method | Avg Val Acc | Avg Val F1 | Total Time |
| -------------- | --------------- | ----------- | ---------- | ---------- |
| Traditional | CLAHE, SVM | 0.3312 | 0.2791 | 39m 5.13s |
| Traditional | 256 resize, blurring, SVM | 0.3857 | 0.3466 | 50m 20.82s |
| Traditional | 256 resize, LBP, SVM | 0.4333 | 0.3794 | 0m 1.71s |
| Traditional | CLAHE, blurring, SVM | 0.3709 | 0.3200 | 38m 39.53s |
| Traditional | CLAHE, LBP, SVM | 0.4125 | 0.3417 | 0m 1.74s |
| Traditional | blurring, LBP, SVM | 0.4397 | 0.4176 | 0m 1.68s |
| Traditional | CLAHE, blurring, LBP, SVM | 0.4342 | 0.3839 | 0m 1.72s |
| Traditional | 256 resize, CLAHE, blurring, LBP, SVM | 0.4406 | 0.3931 | 0m 1.70s |
| Transformer | **DINOv2, SVM** | **0.8471** | **0.8444** | 0m 15.99s |
| Transformer | DINOv2, RF | 0.7658 | 0.7543 | 0m 24.41s |
| CNN         | VGG16, SVM | 0.4129 | 0.3660 | 0m 12.27s |
| CNN         | VGG16, RF | 0.4166 | 0.3522 | 0m 3.81s |

**Conclusion**

When comparing the final average accuracy and f1 scores of the validation sets, it can be seen that using CNN-based preprocessing, specifically using the DINOv2 model, outperformed the traditional methods. As such, feature extraction using the DINOv2 model will be used as the final preprocessing technique.

Furthermore, SVM will be used instead of Random Forest as the classification algorithm as it produced slightly better accuracy and f1 scores and doesn't overfit as much compared to the other methods.

## Hypertuning

To make the SVM perform better, hypertuning will be performed on the train and validation set. The test set will finally be used  to see the metrics and overall performance of the hypertuned model with the chosen preprocessing technique (which is the feature extraction using the DINOv2 model).

In [18]:
def hypertune_SVM(
    X: list[cv_typing.MatLike],
    y: np.ndarray[typing.Any, typing.Any],
    k_folds: int = 5,
    param_grid: dict[str, typing.Any] | None = None,
) -> dict:
    """
    Function to hypertune an SVM using Grid Search CV.
    Returns the best estimator, best parameters, and the results of the cv.
    """

    # Preprocess the image using DinoV2 Feature Extraction
    print("Preprocessing image using feature extraction")
    X_preprocessed = preprocess_img_other(imgs=X, use_vgg=False, use_dino=True)

    print("Done preprocessing images")

    # Define pipeline: scaler + SVM
    pipeline = make_pipeline(
        StandardScaler(),
        SVC()
    )

    # Stratified K Fold
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    # Define Grid Search CV
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring="accuracy",
        cv=skf,
        refit=True,
        verbose=1, # display computation time for each fold and parameter
        n_jobs=-1, # use all processors (run in parallel)
    )

    # Train and fit
    print("Starting Grid Search")
    grid.fit(X_preprocessed, y)

    print(f"\tBest Parameters: {grid.best_params_}")
    print(f"\tScore of Best Estimator: {grid.best_score_}")

    return {
        "best_estimator": grid.best_estimator_,
        "best_params": grid.best_params_,
        "best_score": grid.best_score_,
        "cv_results": grid.cv_results_
    }

### Running the Hypertuning Function

In [19]:
rbf_params = {
    'svc__kernel': ['rbf'],
    'svc__C': [0.01, 0.1, 1, 10, 50],
    'svc__gamma': ['scale', 0.01, 0.001, 0.1]
}

print("Hypertuning using RBF Kernel")
hypertuned_results_rbf = hypertune_SVM(
    X=X_train_val_other,
    y=y_train_val_other,
    k_folds=5,
    param_grid=rbf_params
)

Hypertuning using RBF Kernel
Preprocessing image using feature extraction


  0%|          | 0/271 [00:00<?, ?it/s]

	Procesing batch 0
	Finished processing the batch
	Features taken using  DINOv2


  0%|          | 1/271 [00:00<01:05,  4.10it/s]

	Completed all batches for  DINOv2

	Procesing batch 8
	Finished processing the batch
	Features taken using  DINOv2


...

 99%|█████████▉| 269/271 [01:05<00:00,  4.19it/s]

	Completed all batches for  DINOv2

	Procesing batch 2152
	Finished processing the batch
	Features taken using  DINOv2


100%|██████████| 271/271 [01:05<00:00,  4.14it/s]

	Completed all batches for  DINOv2

	Procesing batch 2160
	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

Done preprocessing images
Starting Grid Search
Fitting 5 folds for each of 20 candidates, totalling 100 fits


	Best Parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
	Score of Best Estimator: 0.8471131639722863


In [20]:
poly_params = {
    'svc__kernel': ['poly'],
    'svc__C': [1, 10],
    'svc__gamma': ['scale'],
    'svc__degree': [2, 3],
    'svc__coef0': [0.0, 1.0],
}

print("Hypertuning using Polynomial Kernel")
hypertuned_results_poly = hypertune_SVM(
    X=X_train_val_other,
    y=y_train_val_other,
    k_folds=5,
    param_grid=poly_params
)

Hypertuning using Polynomial Kernel
Preprocessing image using feature extraction


  0%|          | 0/271 [00:00<?, ?it/s]

	Procesing batch 0
	Finished processing the batch
	Features taken using  DINOv2


  0%|          | 1/271 [00:00<01:04,  4.21it/s]

	Completed all batches for  DINOv2

	Procesing batch 8
	Finished processing the batch
	Features taken using  DINOv2


...

 99%|█████████▉| 269/271 [01:05<00:00,  4.17it/s]

	Completed all batches for  DINOv2

	Procesing batch 2152
	Finished processing the batch
	Features taken using  DINOv2


100%|██████████| 271/271 [01:05<00:00,  4.14it/s]

	Completed all batches for  DINOv2

	Procesing batch 2160
	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

Done preprocessing images
Starting Grid Search
Fitting 5 folds for each of 8 candidates, totalling 40 fits


	Best Parameters: {'svc__C': 1, 'svc__coef0': 1.0, 'svc__degree': 3, 'svc__gamma': 'scale', 'svc__kernel': 'poly'}
	Score of Best Estimator: 0.8508083140877598


In [22]:
print(f"RBF Best Score: {hypertuned_results_rbf['best_score']}")
print(f"Polynomial Best Score: {hypertuned_results_poly['best_score']}")

RBF Best Score: 0.8471131639722863
Polynomial Best Score: 0.8508083140877598


Based on the results, it can be seen that the SVM model using a polynomial kernel of degree 3 produced a higher score for the best estimator than the model using an RBF kernel.

### Testing on Test Set

Using the best hypertuned model, we use it on the test set to get the final metrics for a general idea of how well the model performs.

In [21]:
# Define
final_model = hypertuned_results_poly['best_estimator']

# Preprocess image
print("Preprocessing image using CNN")
X_preprocessed = preprocess_img_other(imgs=X_test_other, use_vgg=False, use_dino=True)
print("Done preprocessing images")

# Predict
y_hyp_pred = final_model.predict(X_preprocessed)

Preprocessing image using CNN


  0%|          | 0/68 [00:00<?, ?it/s]

	Procesing batch 0
	Finished processing the batch
	Features taken using  DINOv2


  1%|▏         | 1/68 [00:00<00:15,  4.22it/s]

	Completed all batches for  DINOv2

	Procesing batch 8
	Finished processing the batch
	Features taken using  DINOv2


...

	Completed all batches for  DINOv2

	Procesing batch 520
	Finished processing the batch
	Features taken using  DINOv2


 97%|█████████▋| 66/68 [00:15<00:00,  4.21it/s]

	Completed all batches for  DINOv2

	Procesing batch 528
	Finished processing the batch
	Features taken using  DINOv2


100%|██████████| 68/68 [00:16<00:00,  4.21it/s]

	Completed all batches for  DINOv2

	Procesing batch 536
	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2

Done preprocessing images


In [29]:
# Print summary
print("=== Metrics ===")
print(f"\tVal Acc: {(accuracy_score(y_test_other, y_hyp_pred)):.4f}")
print(f"\tVal F1: {(f1_score(y_test_other, y_hyp_pred, average='macro')):.4f}")
print("\nClassification Report")
print(classification_report(y_test_other, y_hyp_pred))

=== Metrics ===
	Val Acc: 0.8450
	Val F1: 0.8442

Classification Report
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       103
           1       0.77      0.81      0.79       135
           2       0.82      0.75      0.78        36
           3       0.76      0.76      0.76       110
           4       0.94      0.81      0.87        62
           5       0.85      0.92      0.88        96

    accuracy                           0.85       542
   macro avg       0.86      0.84      0.84       542
weighted avg       0.85      0.85      0.85       542



Comparing it to the "baseline" model (or general model placed in the `run_train_cross_val` function), it can be seen that the hypertuned model performs slightly better than the "baseline" model by less than 1%.

In [24]:
pipeline = make_pipeline(
    StandardScaler(),
    SVC(kernel="rbf", C=10, gamma='scale')
)

X_preprocessed_train = preprocess_img_other(imgs=X_train_val_other, use_vgg=False, use_dino=True)
pipeline.fit(X_preprocessed_train, y_train_val_other)

# Predict
y_base_pred = pipeline.predict(X_preprocessed)

  0%|          | 0/271 [00:00<?, ?it/s]

	Procesing batch 0
	Finished processing the batch
	Features taken using  DINOv2


  0%|          | 1/271 [00:00<01:05,  4.12it/s]

	Completed all batches for  DINOv2

	Procesing batch 8
	Finished processing the batch
	Features taken using  DINOv2


  1%|          | 2/271 [00:00<01:05,  4.13it/s]

...

	Completed all batches for  DINOv2

	Procesing batch 2144
	Finished processing the batch
	Features taken using  DINOv2


 99%|█████████▉| 269/271 [01:05<00:00,  4.19it/s]

	Completed all batches for  DINOv2

	Procesing batch 2152
	Finished processing the batch
	Features taken using  DINOv2


100%|██████████| 271/271 [01:05<00:00,  4.13it/s]

	Completed all batches for  DINOv2

	Procesing batch 2160
	Finished processing the batch
	Features taken using  DINOv2
	Completed all batches for  DINOv2



In [30]:
# Print summary
print("=== Metrics ===")
print(f"\tVal Acc: {(accuracy_score(y_test_other, y_base_pred)):.4f}")
print(f"\tVal F1: {(f1_score(y_test_other, y_base_pred, average='macro')):.4f}")
print("\nClassification Report")
print(classification_report(y_test_other, y_base_pred))

=== Metrics ===
	Val Acc: 0.8413
	Val F1: 0.8411

Classification Report
              precision    recall  f1-score   support

           0       0.99      0.96      0.98       103
           1       0.75      0.84      0.79       135
           2       0.82      0.75      0.78        36
           3       0.76      0.76      0.76       110
           4       0.98      0.77      0.86        62
           5       0.86      0.88      0.87        96

    accuracy                           0.84       542
   macro avg       0.86      0.83      0.84       542
weighted avg       0.85      0.84      0.84       542



## Saving the Model

In [31]:
# Using pickle to save the model as a pickle file
filename = f"{model_dir}/svm_model_final.pkl"
pickle.dump(final_model, open(filename, 'wb'))